### Ensemble trees for kaviar
* duplicate positons are removed
* focus on cgi vs both, limit by ahmad good regions
* focus on high freq vars?

In [23]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
from sklearn import tree
%matplotlib inline

In [43]:
kaviar_mat_file = '../data/interim/kaviar.mat'
dat = pd.read_csv(kaviar_mat_file, sep='\t')

/Users/perry/miniconda3/envs/notebook/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [44]:
# limit to not rare
dat = dat[dat.af>2**-14]
features = [x for x in dat.columns if not x in ('kaviar_status', 'var_type', 'chrom', 'pos', 'ref', 'alt')]
dat = dat.drop_duplicates(subset=['chrom', 'pos'])

In [6]:
dat_cgi = dat[(dat['ahmad_status'] == 1) & ((dat.kaviar_status=='cgi') | (dat.kaviar_status=='both'))]
dat_cgi.loc[:, 'y'] = dat_cgi.apply(lambda row: 1 if row['kaviar_status'] == 'both' else 0, axis=1)
X = dat_cgi[features].values
X = StandardScaler().fit_transform(X)
y = dat_cgi['y'].values

rfc = RandomForestClassifier(n_estimators=200)
clf = svm.SVC(kernel='poly', C=1)
scores = cross_val_score(clf, X, y, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

/Users/perry/miniconda3/envs/notebook/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/perry/miniconda3/envs/notebook/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/perry/miniconda3/envs/notebook/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings

Accuracy: 0.65 (+/- 0.04)


In [5]:
dat_cgi.groupby('kaviar_status').size()

kaviar_status
both    12260
cgi      6997
dtype: int64

In [7]:
dat_cgi = dat[(dat['ahmad_status'] == 0) & ((dat.kaviar_status=='cgi') | (dat.kaviar_status=='both'))]
dat_cgi.loc[:, 'y'] = dat_cgi.apply(lambda row: 1 if row['kaviar_status'] == 'both' else 0, axis=1)
X = dat_cgi[features].values
X = StandardScaler().fit_transform(X)
y = dat_cgi['y'].values

rfc = RandomForestClassifier(n_estimators=200)
clf = svm.SVC(kernel='poly', C=1)
scores = cross_val_score(clf, X, y, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

/Users/perry/miniconda3/envs/notebook/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/perry/miniconda3/envs/notebook/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/perry/miniconda3/envs/notebook/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings

Accuracy: 0.69 (+/- 0.13)


In [8]:
dat_cgi.groupby('kaviar_status').size()

kaviar_status
both    418
cgi     918
dtype: int64

In [9]:
dat_ill = dat[(dat['ahmad_status'] == 0) & ((dat.kaviar_status=='ill') | (dat.kaviar_status=='both'))]
dat_ill.groupby('kaviar_status').size()

kaviar_status
both     418
ill     4218
dtype: int64

In [10]:
dat_ill = dat[(dat['ahmad_status'] == 1) & ((dat.kaviar_status=='ill') | (dat.kaviar_status=='both'))]
dat_ill.groupby('kaviar_status').size()

kaviar_status
both     12260
ill     162713
dtype: int64

In [11]:
(162713/12260) / (4218/418)

1.3152293402701232

In [12]:
(6997/12260) / (918/418)

0.25986931658652224

In [20]:
features

['lowmappabilityall',
 'notinlowmappabilityall',
 'siren_similarRegions_dist1',
 'segdupall',
 'notinsegdupall',
 'notinrefseq_union_cds.sort',
 'BadPromoters_gb-2013-14-5-r51-s1',
 'human_g1k_v37_l100_gclt30orgt55_slop50',
 'human_g1k_v37_l100_gc30to55_slop50',
 'hg19_self_chain_split.sort',
 'hg19_self_chain_split_both',
 'notinAllRepeats_gt95percidentity_slop5',
 'AllRepeats_gt95percidentity_slop5',
 'AllRepeats_lt51bp_gt95identity_merged',
 'ahmad_status']

In [45]:
dat_cgi = dat[((dat['ahmad_status'] == 1) | (dat['ahmad_status'] == 0)) & ((dat.kaviar_status=='cgi') | (dat.kaviar_status=='both'))]
dat_cgi.loc[:, 'y'] = dat_cgi.apply(lambda row: 1 if row['kaviar_status'] == 'both' else 0, axis=1)
dat_cgi.groupby('y').size()

/Users/perry/miniconda3/envs/notebook/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/perry/miniconda3/envs/notebook/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


y
0     3524
1    42601
dtype: int64

In [46]:
#dat_cgi = dat[((dat['ahmad_status'] == 1) | (dat['ahmad_status'] == 0)) & ((dat.kaviar_status=='cgi') | (dat.kaviar_status=='both'))]
#dat_cgi.loc[:, 'y'] = dat_cgi.apply(lambda row: 1 if row['kaviar_status'] == 'both' else 0, axis=1)

pos_cgi = dat_cgi[dat_cgi.y==1].sample(3500)
neg_cgi = dat_cgi[dat_cgi.y==0].sample(3500)
dat_cgi = pd.concat([pos_cgi, neg_cgi])

X = dat_cgi[features].values
X = StandardScaler().fit_transform(X)
y = dat_cgi['y'].values

rfc = RandomForestClassifier(n_estimators=200)
clf = svm.SVC(kernel='poly', C=1)
clf_tree = tree.DecisionTreeClassifier(max_depth=2)
scores = cross_val_score(rfc, X, y, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

dat_cgi.groupby('kaviar_status').size()

Accuracy: 0.76 (+/- 0.04)


kaviar_status
both    3500
cgi     3500
dtype: int64